## Setup

In [1]:
import os

REPO_URL = "https://github.com/fabioantonacci79/BasicDSP.git"
REPO_NAME = "BasicDSP"

if not os.path.exists(REPO_NAME):
    !git clone {REPO_URL}

%cd BasicDSP/notebooks

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import remez, freqz
import ipywidgets as widgets
from IPython.display import display

# ---------- Helpers ----------
def db_to_delta(db):
    """Convert attenuation in dB to linear ripple delta."""
    return 10 ** (-db / 20.0)

def passband_ripple_db_to_delta(rp_db):
    """
    Convert passband ripple in dB (peak-to-peak in magnitude, approx) to delta_p.
    For teaching: treat Rp as ± ripple around 1 in dB.
    """
    # +/- rp_db around 0 dB -> upper gain = 10^(rp/20), lower = 10^(-rp/20)
    # delta_p ~ (upper - lower)/2 around 1; use symmetric approximation
    upper = 10 ** (rp_db / 20.0)
    lower = 10 ** (-rp_db / 20.0)
    return (upper - lower) / 2.0

def design_equiripple_fir(filter_type, numtaps, bands, desired, weights, grid_density=16, fs=2.0):
    """
    Remez (Parks–McClellan) equiripple FIR.
    fs=2.0 => frequency bands expressed on [0,1] (Nyquist=1).
    """
    # SciPy remez expects bands increasing and within [0, fs/2] = [0,1]
    h = remez(numtaps, bands, desired, weight=weights, fs=fs, grid_density=grid_density)
    return h

def freq_response(h, worN=2048):
    # Use fs=2 => normalized digital rad freq mapping to f=omega/pi in [0,1]
    w, H = freqz(h, worN=worN, whole=False)  # w in [0, pi]
    f = w / np.pi
    mag = np.abs(H)
    return f, mag

def plot_spec_and_response(ax_spec, ax_freq, ax_time, f, mag, h,
                           band_edges, desired_func, dp, ds,
                           title_prefix=""):
    """
    ax_spec: desired + tolerance "window" visualization
    ax_freq: magnitude + discrete samples
    ax_time: impulse response
    """
    # ---- Spec plot ("filter window") ----
    ax_spec.clear()
    ff = np.linspace(0, 1, 1500)
    dd = desired_func(ff)

    ax_spec.plot(ff, dd)  # desired
    # tolerance bands (only meaningful in pass/stop regions)
    # We'll draw simple envelopes for the whole axis (teaching-friendly).
    ax_spec.plot(ff, np.clip(dd + (dp if dd.max() > 0 else ds), 0, 1.2))
    ax_spec.plot(ff, np.clip(dd - (dp if dd.max() > 0 else ds), -0.2, 1.2))

    # draw band edges
    for be in band_edges:
        ax_spec.axvline(be, linewidth=1)

    ax_spec.set_xlim(0, 1)
    ax_spec.set_ylim(-0.1, 1.15)
    ax_spec.grid(True)
    ax_spec.set_xlabel(r"Normalized frequency $f=\omega/\pi$")
    ax_spec.set_ylabel("Magnitude")
    ax_spec.set_title(f"{title_prefix}Design mask (desired + tolerance)")

    # ---- Frequency response (continuous + samples) ----
    ax_freq.clear()
    ax_freq.plot(f, mag)

    # sampled points for "samples in frequency"
    fsamp = np.linspace(0, 1, 64)
    mags = np.interp(fsamp, f, mag)
    ax_freq.plot(fsamp, mags, marker='o', linestyle='None')  # points

    for be in band_edges:
        ax_freq.axvline(be, linewidth=1)

    ax_freq.set_xlim(0, 1)
    ax_freq.set_ylim(-0.1, 1.15)
    ax_freq.grid(True)
    ax_freq.set_xlabel(r"Normalized frequency")
    ax_freq.set_ylabel(r"$|H(e^{j\omega})|$")
    ax_freq.set_title(f"{title_prefix}Magnitude response + sampled points")

    # ---- Time domain ----
    ax_time.clear()
    n = np.arange(len(h))
    ax_time.stem(n, h)
    ax_time.grid(True)
    ax_time.set_xlabel("n")
    ax_time.set_ylabel("h[n]")
    ax_time.set_title(f"{title_prefix}Impulse response (time domain)")

    plt.tight_layout()


/Users/fabioantonacci/Documents/GitHub/BasicDSP/notebooks/BasicDSP/notebooks


## 1. Lowpass filter

In [2]:
def lowpass_ui():
    fc = widgets.FloatSlider(value=0.35, min=0.05, max=0.95, step=0.01, description="cutoff fc")
    df = widgets.FloatSlider(value=0.10, min=0.02, max=0.40, step=0.01, description="transition Δf")
    rp = widgets.FloatSlider(value=0.2, min=0.01, max=2.0, step=0.01, description="Rp (dB)")
    As = widgets.FloatSlider(value=60, min=20, max=100, step=1, description="As (dB)")
    taps = widgets.IntSlider(value=61, min=11, max=401, step=2, description="numtaps")

    out = widgets.Output(layout={"border": "1px solid #ddd", "padding": "6px"})

    def update(change=None):
        with out:
            out.clear_output(wait=True)
            try:
                # band edges
                fp = fc.value - df.value / 2
                fs = fc.value + df.value / 2

                # keep away from exact 0/1 to avoid edge-case numeric issues
                fp = float(np.clip(fp, 0.01, 0.99))
                fs = float(np.clip(fs, 0.01, 0.99))

                if not (fp < fs):
                    print("Invalid bands: need fp < fs. Increase Δf or move fc.")
                    return

                dp = passband_ripple_db_to_delta(rp.value)
                ds = db_to_delta(As.value)

                Wp = 1.0
                Ws = float(dp / ds)

                bands = [0.0, fp, fs, 1.0]
                desired = [1.0, 0.0]
                weights = [Wp, Ws]

                h = design_equiripple_fir("lp", taps.value, bands, desired, weights, grid_density=16, fs=2.0)
                f, mag = freq_response(h)

                fig, axs = plt.subplots(3, 1, figsize=(10, 10))
                plot_spec_and_response(
                    axs[0], axs[1], axs[2],
                    f, mag, h,
                    band_edges=[fp, fs],
                    desired_func=lambda ff: np.where(ff <= fp, 1.0, np.where(ff >= fs, 0.0, 0.5)),
                    dp=dp, ds=ds,
                    title_prefix="Lowpass: "
                )
                plt.show()
                plt.close(fig)

                print(f"Passband edge fp={fp:.3f}, Stopband edge fs={fs:.3f} (normalized)")
                print(f"delta_p≈{dp:.4f}, delta_s≈{ds:.6f}, weights: Wp={Wp:.2f}, Ws={Ws:.2f}")

            except Exception as e:
                print("Error while updating lowpass design:")
                print(repr(e))

    for w in (fc, df, rp, As, taps):
        w.observe(update, "value")

    update()
    controls = widgets.VBox([widgets.HBox([fc, df]), widgets.HBox([rp, As]), taps])
    return widgets.VBox([controls, out])

lp_widget = lowpass_ui()
display(lp_widget)


## 2. Highpass filter design

In [3]:
def highpass_ui():
    fc = widgets.FloatSlider(value=0.45, min=0.05, max=0.95, step=0.01, description="cutoff fc")
    df = widgets.FloatSlider(value=0.10, min=0.02, max=0.40, step=0.01, description="transition Δf")
    rp = widgets.FloatSlider(value=0.2, min=0.01, max=2.0, step=0.01, description="Rp (dB)")
    As = widgets.FloatSlider(value=60, min=20, max=100, step=1, description="As (dB)")
    taps = widgets.IntSlider(value=61, min=11, max=401, step=2, description="numtaps")

    out = widgets.Output(layout={"border": "1px solid #ddd", "padding": "6px"})

    def update(change=None):
        with out:
            out.clear_output(wait=True)
            try:
                # highpass edges: stop [0, fs], pass [fp, 1]
                fs = fc.value - df.value / 2
                fp = fc.value + df.value / 2

                fs = float(np.clip(fs, 0.01, 0.99))
                fp = float(np.clip(fp, 0.01, 0.99))

                if not (fs < fp):
                    print("Invalid bands: need fs < fp. Increase Δf or move fc.")
                    return

                dp = passband_ripple_db_to_delta(rp.value)
                ds = db_to_delta(As.value)

                Wp = 1.0
                Ws = float(dp / ds)

                bands = [0.0, fs, fp, 1.0]
                desired = [0.0, 1.0]
                weights = [Ws, Wp]

                h = design_equiripple_fir("hp", taps.value, bands, desired, weights, grid_density=16, fs=2.0)
                f, mag = freq_response(h)

                fig, axs = plt.subplots(3, 1, figsize=(10, 10))
                plot_spec_and_response(
                    axs[0], axs[1], axs[2],
                    f, mag, h,
                    band_edges=[fs, fp],
                    desired_func=lambda ff: np.where(ff <= fs, 0.0, np.where(ff >= fp, 1.0, 0.5)),
                    dp=dp, ds=ds,
                    title_prefix="Highpass: "
                )
                plt.show()
                plt.close(fig)

                print(f"Stopband edge fs={fs:.3f}, Passband edge fp={fp:.3f} (normalized)")
                print(f"delta_p≈{dp:.4f}, delta_s≈{ds:.6f}, weights: Wp={Wp:.2f}, Ws={Ws:.2f}")

            except Exception as e:
                print("Error while updating highpass design:")
                print(repr(e))

    for w in (fc, df, rp, As, taps):
        w.observe(update, "value")

    update()
    controls = widgets.VBox([widgets.HBox([fc, df]), widgets.HBox([rp, As]), taps])
    return widgets.VBox([controls, out])
hp_widget = highpass_ui()
display(hp_widget)

## 3. Bandpass filter design

In [4]:
def bandpass_ui():
    f0 = widgets.FloatSlider(value=0.40, min=0.05, max=0.95, step=0.01, description="center f0")
    bpw = widgets.FloatSlider(value=0.20, min=0.02, max=0.80, step=0.01, description="PB width")
    df = widgets.FloatSlider(value=0.08, min=0.02, max=0.30, step=0.01, description="transition Δf")
    rp = widgets.FloatSlider(value=0.2, min=0.01, max=2.0, step=0.01, description="Rp (dB)")
    As = widgets.FloatSlider(value=60, min=20, max=100, step=1, description="As (dB)")
    taps = widgets.IntSlider(value=101, min=11, max=401, step=2, description="numtaps")

    out = widgets.Output()

    def update(_=None):
        out.clear_output(wait=True)
        with out:
            # passband edges
            fp1 = f0.value - bpw.value/2
            fp2 = f0.value + bpw.value/2

            # stopband edges around passband with transition df on each side
            fs1 = fp1 - df.value
            fs2 = fp2 + df.value

            # sanity clamp
            if fs1 < 0.0: fs1 = 0.01
            if fp1 <= fs1: fp1 = fs1 + 0.01
            if fp2 >= 1.0: fp2 = 0.99
            if fs2 >= 1.0: fs2 = 0.99
            if fp2 <= fp1:
                print("Invalid bandpass: increase PB width or move center.")
                return
            if fs1 >= fp1 or fp2 >= fs2:
                print("Invalid transitions: reduce Δf or adjust bands.")
                return

            dp = passband_ripple_db_to_delta(rp.value)
            ds = db_to_delta(As.value)

            Wp = 1.0
            Ws = dp / ds

            # Bands: [0, fs1] stop, [fp1, fp2] pass, [fs2, 1] stop
            bands = [0.0, fs1, fp1, fp2, fs2, 1.0]
            desired = [0.0, 1.0, 0.0]
            weights = [Ws, Wp, Ws]

            h = design_equiripple_fir("bp", taps.value, bands, desired, weights, grid_density=16, fs=2.0)
            f, mag = freq_response(h)

            fig, axs = plt.subplots(3, 1, figsize=(10, 10))
            plot_spec_and_response(
                axs[0], axs[1], axs[2],
                f, mag, h,
                band_edges=[fs1, fp1, fp2, fs2],
                desired_func=lambda ff: np.where((ff >= fp1) & (ff <= fp2), 1.0, 0.0),
                dp=dp, ds=ds,
                title_prefix="Bandpass: "
            )
            plt.show()

            print(f"Edges (normalized): fs1={fs1:.3f}, fp1={fp1:.3f}, fp2={fp2:.3f}, fs2={fs2:.3f}")
            print(f"delta_p≈{dp:.4f}, delta_s≈{ds:.6f}, weights: Wp={Wp:.2f}, Ws={Ws:.2f}")

    for w in (f0, bpw, df, rp, As, taps):
        w.observe(update, "value")

    update()
    controls = widgets.VBox([widgets.HBox([f0, bpw, df]), widgets.HBox([rp, As]), taps])
    return widgets.VBox([controls, out])

bp_widget = bandpass_ui()
display(bp_widget)


## 4. Application of filters to audio signal

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import lfilter
import ipywidgets as widgets
from IPython.display import display, Audio

AUDIO_DIR = "/content/BasicDSP/notebooks/audio_dataset"   # <-- change if needed


def list_wavs(audio_dir=AUDIO_DIR):
    if not os.path.isdir(audio_dir):
        return []
    return sorted([f for f in os.listdir(audio_dir) if f.lower().endswith(".wav")])


def read_wav_mono(path):
    fs, x = wavfile.read(path)

    # Convert to float in [-1, 1] approximately
    if x.dtype == np.int16:
        x = x.astype(np.float32) / 32768.0
    elif x.dtype == np.int32:
        x = x.astype(np.float32) / 2147483648.0
    elif x.dtype == np.uint8:
        x = (x.astype(np.float32) - 128.0) / 128.0
    else:
        x = x.astype(np.float32)

    # Mono
    if x.ndim == 2:
        x = x.mean(axis=1)

    return fs, x


def apply_fir(x, h):
    # FIR filtering (causal). For demos, this is fine.
    y = lfilter(h, [1.0], x)
    # optional: compensate group delay for listening (linear-phase FIR delay = (N-1)/2)
    gd = (len(h) - 1) // 2
    if gd > 0 and len(y) > gd:
        y = y[gd:]
        x2 = x[:len(y)]
    else:
        x2 = x[:len(y)]
    return x2, y


def audio_filtering_demo():
    wavs = list_wavs(AUDIO_DIR)
    if not wavs:
        print(f"No .wav files found in {AUDIO_DIR}. Put your dataset wavs there or change AUDIO_DIR.")
        return

    file_dd = widgets.Dropdown(options=wavs, description="Audio file")

    ftype = widgets.ToggleButtons(
        options=["Lowpass", "Highpass", "Bandpass"],
        value="Lowpass",
        description="Filter"
    )

    # Common controls
    rp = widgets.FloatSlider(value=0.2, min=0.01, max=2.0, step=0.01, description="Rp (dB)")
    As = widgets.FloatSlider(value=60, min=20, max=100, step=1, description="As (dB)")
    taps = widgets.IntSlider(value=101, min=11, max=401, step=2, description="numtaps")

    # LP/HP controls
    fc = widgets.FloatSlider(value=0.30, min=0.02, max=0.98, step=0.01, description="fc (×Nyq)")
    df = widgets.FloatSlider(value=0.10, min=0.02, max=0.50, step=0.01, description="Δf (×Nyq)")

    # BP controls
    f0 = widgets.FloatSlider(value=0.40, min=0.05, max=0.95, step=0.01, description="f0 (×Nyq)")
    bpw = widgets.FloatSlider(value=0.20, min=0.02, max=0.90, step=0.01, description="PB width")
    df_bp = widgets.FloatSlider(value=0.08, min=0.02, max=0.30, step=0.01, description="Δf (×Nyq)")

    out = widgets.Output(layout={"border": "1px solid #ddd", "padding": "8px"})

    # show/hide parameter groups
    lp_hp_box = widgets.VBox([widgets.HBox([fc, df])])
    bp_box = widgets.VBox([widgets.HBox([f0, bpw, df_bp])])

    def refresh_visibility():
        if ftype.value in ["Lowpass", "Highpass"]:
            lp_hp_box.layout.display = ""
            bp_box.layout.display = "none"
        else:
            lp_hp_box.layout.display = "none"
            bp_box.layout.display = ""

    def update(_=None):
        refresh_visibility()
        with out:
            out.clear_output(wait=True)
            try:
                # ----- load audio -----
                path = os.path.join(AUDIO_DIR, file_dd.value)
                fs_audio, x = read_wav_mono(path)

                # normalize frequency parameters in [0,1] where 1 = Nyquist
                # We'll design using the same convention as before: bands in [0,1], using remez(fs=2.0).
                dp = passband_ripple_db_to_delta(rp.value)
                ds = db_to_delta(As.value)
                Wp = 1.0
                Ws = float(dp / ds)

                if ftype.value == "Lowpass":
                    fp = float(np.clip(fc.value - df.value/2, 0.01, 0.99))
                    fsb = float(np.clip(fc.value + df.value/2, 0.01, 0.99))
                    if fp >= fsb:
                        print("Invalid lowpass bands: need fp < fs. Increase Δf or move fc.")
                        return
                    bands = [0.0, fp, fsb, 1.0]
                    desired = [1.0, 0.0]
                    weights = [Wp, Ws]
                    band_edges = [fp, fsb]
                    title = f"Lowpass (fp={fp:.2f}, fs={fsb:.2f})"

                elif ftype.value == "Highpass":
                    fsb = float(np.clip(fc.value - df.value/2, 0.01, 0.99))
                    fp = float(np.clip(fc.value + df.value/2, 0.01, 0.99))
                    if fsb >= fp:
                        print("Invalid highpass bands: need fs < fp. Increase Δf or move fc.")
                        return
                    bands = [0.0, fsb, fp, 1.0]
                    desired = [0.0, 1.0]
                    weights = [Ws, Wp]
                    band_edges = [fsb, fp]
                    title = f"Highpass (fs={fsb:.2f}, fp={fp:.2f})"

                else:  # Bandpass
                    fp1 = f0.value - bpw.value/2
                    fp2 = f0.value + bpw.value/2
                    fs1 = fp1 - df_bp.value
                    fs2 = fp2 + df_bp.value

                    fs1 = float(np.clip(fs1, 0.01, 0.99))
                    fp1 = float(np.clip(fp1, 0.01, 0.99))
                    fp2 = float(np.clip(fp2, 0.01, 0.99))
                    fs2 = float(np.clip(fs2, 0.01, 0.99))

                    if not (fs1 < fp1 < fp2 < fs2):
                        print("Invalid bandpass edges: need fs1 < fp1 < fp2 < fs2.")
                        return

                    bands = [0.0, fs1, fp1, fp2, fs2, 1.0]
                    desired = [0.0, 1.0, 0.0]
                    weights = [Ws, Wp, Ws]
                    band_edges = [fs1, fp1, fp2, fs2]
                    title = f"Bandpass (fp1={fp1:.2f}, fp2={fp2:.2f})"

                # ----- design filter -----
                h = design_equiripple_fir("audio", taps.value, bands, desired, weights, grid_density=16, fs=2.0)
                f, mag = freq_response(h)

                # ----- apply to audio -----
                x_aligned, y = apply_fir(x, h)

                # ----- players -----
                print(f"File: {file_dd.value} | fs={fs_audio} Hz | Filter: {title}")
                display(widgets.HTML("<b>Original</b>"))
                display(Audio(x_aligned, rate=fs_audio))
                display(widgets.HTML("<b>Filtered</b>"))
                display(Audio(y, rate=fs_audio))

                # ----- plots -----
                fig, axs = plt.subplots(2, 1, figsize=(11, 7))

                # waveform (first ~50 ms or shorter if tiny)
                Nshow = int(min(len(x_aligned), fs_audio * 0.05))
                t = np.arange(Nshow) / fs_audio
                axs[0].plot(t, x_aligned[:Nshow], label="original")
                axs[0].plot(t, y[:Nshow], label="filtered")
                axs[0].set_xlabel("Time (s)")
                axs[0].set_ylabel("Amplitude")
                axs[0].set_title("Waveform (first 50 ms, aligned for group delay)")
                axs[0].grid(True)
                axs[0].legend()

                # magnitude response
                axs[1].plot(f, mag)
                for be in band_edges:
                    axs[1].axvline(be, linewidth=1)
                axs[1].set_xlim(0, 1)
                axs[1].set_ylim(-0.05, 1.15)
                axs[1].set_xlabel(r"Normalized frequency $f=\omega/\pi$ (1 = Nyquist)")
                axs[1].set_ylabel(r"$|H(e^{j\omega})|$")
                axs[1].set_title("Designed FIR magnitude response (minimax / equiripple)")
                axs[1].grid(True)

                plt.tight_layout()
                plt.show()
                plt.close(fig)

                print(f"delta_p≈{dp:.4f}, delta_s≈{ds:.6f}, weights: Wp={Wp:.2f}, Ws={Ws:.2f}")

            except Exception as e:
                print("Error in audio filtering demo:")
                print(repr(e))

    for w in [file_dd, ftype, rp, As, taps, fc, df, f0, bpw, df_bp]:
        w.observe(update, "value")

    refresh_visibility()
    update()

    controls = widgets.VBox([
        widgets.HBox([file_dd, ftype]),
        widgets.HBox([rp, As]),
        taps,
        lp_hp_box,
        bp_box,
    ])
    display(widgets.VBox([controls, out]))

audio_filtering_demo()
